## Kafka Producer

In [14]:
import time
import json
import kafka

In [15]:
import io
import avro.schema
from avro.io import DatumWriter

In [16]:
# Without Schema
# topic = 'hello'
# bootstrap_server = "localhost:9092"
# producer = kafka.KafkaProducer(
#     bootstrap_servers=[bootstrap_server],
#     key_serializer=lambda key: key.encode('utf-8'),
#     value_serializer=lambda x: json.dumps(x).encode('utf-8')
# )

In [17]:
# With Schema
"""https://github.com/thanhson1085/python-kafka-avro"""
SCHEMA_PATH = "stock_schema.avsc"
SCHEMA = avro.schema.parse(open(SCHEMA_PATH).read())

In [18]:
topic = 'hello'
bootstrap_server = "localhost:9092"
producer = kafka.KafkaProducer(
    bootstrap_servers=[bootstrap_server]
)

In [19]:
'''
Sample loop to send Data to kafka
for i in range(1000):
    data = {
        "num": i
    }
    producer.send(topic, key="num", value=data)
    time.sleep(3)
'''

'\nSample loop to send Data to kafka\nfor i in range(1000):\n    data = {\n        "num": i\n    }\n    producer.send(topic, key="num", value=data)\n    time.sleep(3)\n'

## Get Stock Live Data

In [20]:
# !pip install yfinance

In [21]:
import yfinance as yf
import datetime

In [22]:
# Getting wierd graphs since there a gap between days
# df = yf.download(tickers='UBER', period='5d', interval='5m')

df = yf.download(tickers='UBER', period='1d', interval='1m')

[*********************100%***********************]  1 of 1 completed


In [23]:
stock_dict = df["High"].to_dict().items()

In [24]:
# for d in stock_dict:
#     data = (d[0].strftime('%Y-%m-%d %H:%M:%S'), d[1])
#     print(data)
#     break

In [25]:
# Without Schema
# try:
#     for d in stock_dict:
#         data = (d[0].strftime('%Y-%m-%d %H:%M:%S'), d[1])
        
#         # We are applying Synchronous send since it is financial data.
#         # We will wait till we received the acknowledgement
#         recorded_metadata = producer.send(topic, key="UBER", value=data).get(timeout=10)
#         print("Below are the data sent: ")
#         print(key, data)
#         print(recorded_metadata.topic)
#         print(recorded_metadata.partition)
#         print(recorded_metadata.offset)
#     #     time.sleep(3)
# except Exception as e:
#     print("We recevied a error:::")
#     print(e)
# finally:
#     print("<---------------->")

In [29]:
# With Schema
try:
    for d in stock_dict:
        
        bytes_writer = io.BytesIO()
        encoder = avro.io.BinaryEncoder(bytes_writer)
        
        writer = DatumWriter(SCHEMA)
        data = {
            "name": "UBER",
            "time": d[0].strftime('%Y-%m-%d %H:%M:%S'),
            "price": d[1]
        }
        writer.write(data, encoder)
        
        raw_bytes = bytes_writer.getvalue()
        print(data, raw_bytes)
        producer.send(topic, raw_bytes)
        time.sleep(1)
except Exception as e:
    print("We recevied a error:::")
    print(e)
finally:
    print("<---------------->")

{'name': 'UBER', 'time': '2023-04-14 09:30:00', 'price': 31.3799991607666} b'\x08UBER&2023-04-14 09:30:00=\n\xfbA'
{'name': 'UBER', 'time': '2023-04-14 09:31:00', 'price': 31.280000686645508} b'\x08UBER&2023-04-14 09:31:00q=\xfaA'
{'name': 'UBER', 'time': '2023-04-14 09:32:00', 'price': 31.239999771118164} b'\x08UBER&2023-04-14 09:32:00\x85\xeb\xf9A'
{'name': 'UBER', 'time': '2023-04-14 09:33:00', 'price': 31.315000534057617} b'\x08UBER&2023-04-14 09:33:00\x1f\x85\xfaA'
{'name': 'UBER', 'time': '2023-04-14 09:34:00', 'price': 31.350000381469727} b'\x08UBER&2023-04-14 09:34:00\xcd\xcc\xfaA'
{'name': 'UBER', 'time': '2023-04-14 09:35:00', 'price': 31.43000030517578} b'\x08UBER&2023-04-14 09:35:00\xa4p\xfbA'
{'name': 'UBER', 'time': '2023-04-14 09:36:00', 'price': 31.489999771118164} b'\x08UBER&2023-04-14 09:36:00\x85\xeb\xfbA'
{'name': 'UBER', 'time': '2023-04-14 09:37:00', 'price': 31.479999542236328} b'\x08UBER&2023-04-14 09:37:00\n\xd7\xfbA'
{'name': 'UBER', 'time': '2023-04-14 09:38:

{'name': 'UBER', 'time': '2023-04-14 10:40:00', 'price': 31.3799991607666} b'\x08UBER&2023-04-14 10:40:00=\n\xfbA'
{'name': 'UBER', 'time': '2023-04-14 10:41:00', 'price': 31.344999313354492} b'\x08UBER&2023-04-14 10:41:00\x8f\xc2\xfaA'
{'name': 'UBER', 'time': '2023-04-14 10:42:00', 'price': 31.30500030517578} b'\x08UBER&2023-04-14 10:42:00\xa4p\xfaA'
{'name': 'UBER', 'time': '2023-04-14 10:43:00', 'price': 31.28499984741211} b'\x08UBER&2023-04-14 10:43:00\xaeG\xfaA'
{'name': 'UBER', 'time': '2023-04-14 10:44:00', 'price': 31.31999969482422} b'\x08UBER&2023-04-14 10:44:00\\\x8f\xfaA'
{'name': 'UBER', 'time': '2023-04-14 10:45:00', 'price': 31.327699661254883} b'\x08UBER&2023-04-14 10:45:00!\x9f\xfaA'
{'name': 'UBER', 'time': '2023-04-14 10:46:00', 'price': 31.290000915527344} b'\x08UBER&2023-04-14 10:46:00\xecQ\xfaA'
{'name': 'UBER', 'time': '2023-04-14 10:47:00', 'price': 31.329999923706055} b'\x08UBER&2023-04-14 10:47:00\xd7\xa3\xfaA'
{'name': 'UBER', 'time': '2023-04-14 10:48:00', 

{'name': 'UBER', 'time': '2023-04-14 11:49:00', 'price': 31.020000457763672} b'\x08UBER&2023-04-14 11:49:00\xf6(\xf8A'
{'name': 'UBER', 'time': '2023-04-14 11:50:00', 'price': 31.010000228881836} b'\x08UBER&2023-04-14 11:50:00{\x14\xf8A'
{'name': 'UBER', 'time': '2023-04-14 11:51:00', 'price': 31.0} b'\x08UBER&2023-04-14 11:51:00\x00\x00\xf8A'
{'name': 'UBER', 'time': '2023-04-14 11:52:00', 'price': 30.989999771118164} b'\x08UBER&2023-04-14 11:52:00\x85\xeb\xf7A'
{'name': 'UBER', 'time': '2023-04-14 11:53:00', 'price': 31.014999389648438} b'\x08UBER&2023-04-14 11:53:00\xb8\x1e\xf8A'
{'name': 'UBER', 'time': '2023-04-14 11:54:00', 'price': 31.014999389648438} b'\x08UBER&2023-04-14 11:54:00\xb8\x1e\xf8A'
{'name': 'UBER', 'time': '2023-04-14 11:55:00', 'price': 31.00589942932129} b'\x08UBER&2023-04-14 11:55:00\x15\x0c\xf8A'
{'name': 'UBER', 'time': '2023-04-14 11:56:00', 'price': 30.9950008392334} b'\x08UBER&2023-04-14 11:56:00\xc3\xf5\xf7A'
{'name': 'UBER', 'time': '2023-04-14 11:57:00',

{'name': 'UBER', 'time': '2023-04-14 12:59:00', 'price': 31.0} b'\x08UBER&2023-04-14 12:59:00\x00\x00\xf8A'
{'name': 'UBER', 'time': '2023-04-14 13:00:00', 'price': 31.024999618530273} b'\x08UBER&2023-04-14 13:00:0033\xf8A'
{'name': 'UBER', 'time': '2023-04-14 13:01:00', 'price': 31.030000686645508} b'\x08UBER&2023-04-14 13:01:00q=\xf8A'
{'name': 'UBER', 'time': '2023-04-14 13:02:00', 'price': 31.059999465942383} b'\x08UBER&2023-04-14 13:02:00\xe1z\xf8A'
{'name': 'UBER', 'time': '2023-04-14 13:03:00', 'price': 31.05500030517578} b'\x08UBER&2023-04-14 13:03:00\xa4p\xf8A'
{'name': 'UBER', 'time': '2023-04-14 13:04:00', 'price': 31.06999969482422} b'\x08UBER&2023-04-14 13:04:00\\\x8f\xf8A'
{'name': 'UBER', 'time': '2023-04-14 13:05:00', 'price': 31.09000015258789} b'\x08UBER&2023-04-14 13:05:00R\xb8\xf8A'
{'name': 'UBER', 'time': '2023-04-14 13:06:00', 'price': 31.09000015258789} b'\x08UBER&2023-04-14 13:06:00R\xb8\xf8A'
{'name': 'UBER', 'time': '2023-04-14 13:07:00', 'price': 31.06999969

{'name': 'UBER', 'time': '2023-04-14 14:09:00', 'price': 31.059999465942383} b'\x08UBER&2023-04-14 14:09:00\xe1z\xf8A'
{'name': 'UBER', 'time': '2023-04-14 14:10:00', 'price': 31.075000762939453} b'\x08UBER&2023-04-14 14:10:00\x9a\x99\xf8A'
{'name': 'UBER', 'time': '2023-04-14 14:11:00', 'price': 31.079999923706055} b'\x08UBER&2023-04-14 14:11:00\xd7\xa3\xf8A'
{'name': 'UBER', 'time': '2023-04-14 14:12:00', 'price': 31.06999969482422} b'\x08UBER&2023-04-14 14:12:00\\\x8f\xf8A'
{'name': 'UBER', 'time': '2023-04-14 14:13:00', 'price': 31.086599349975586} b'\x08UBER&2023-04-14 14:13:00[\xb1\xf8A'
{'name': 'UBER', 'time': '2023-04-14 14:14:00', 'price': 31.100000381469727} b'\x08UBER&2023-04-14 14:14:00\xcd\xcc\xf8A'
{'name': 'UBER', 'time': '2023-04-14 14:15:00', 'price': 31.125} b'\x08UBER&2023-04-14 14:15:00\x00\x00\xf9A'
{'name': 'UBER', 'time': '2023-04-14 14:16:00', 'price': 31.1299991607666} b'\x08UBER&2023-04-14 14:16:00=\n\xf9A'
{'name': 'UBER', 'time': '2023-04-14 14:17:00', 'pri

{'name': 'UBER', 'time': '2023-04-14 15:19:00', 'price': 31.260000228881836} b'\x08UBER&2023-04-14 15:19:00{\x14\xfaA'
{'name': 'UBER', 'time': '2023-04-14 15:20:00', 'price': 31.239999771118164} b'\x08UBER&2023-04-14 15:20:00\x85\xeb\xf9A'
{'name': 'UBER', 'time': '2023-04-14 15:21:00', 'price': 31.226699829101562} b'\x08UBER&2023-04-14 15:21:00H\xd0\xf9A'
{'name': 'UBER', 'time': '2023-04-14 15:22:00', 'price': 31.239999771118164} b'\x08UBER&2023-04-14 15:22:00\x85\xeb\xf9A'
{'name': 'UBER', 'time': '2023-04-14 15:23:00', 'price': 31.270000457763672} b'\x08UBER&2023-04-14 15:23:00\xf6(\xfaA'
{'name': 'UBER', 'time': '2023-04-14 15:24:00', 'price': 31.290000915527344} b'\x08UBER&2023-04-14 15:24:00\xecQ\xfaA'
{'name': 'UBER', 'time': '2023-04-14 15:25:00', 'price': 31.309999465942383} b'\x08UBER&2023-04-14 15:25:00\xe1z\xfaA'
{'name': 'UBER', 'time': '2023-04-14 15:26:00', 'price': 31.30500030517578} b'\x08UBER&2023-04-14 15:26:00\xa4p\xfaA'
{'name': 'UBER', 'time': '2023-04-14 15:27:

In [ ]:
producer.flush()
producer.close()